# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [33]:
orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [13]:
agg_expense = orders.groupby(['CustomerID']).agg({'amount_spent':'sum'})
agg_expense.sort_values('amount_spent', ascending=False)

,amount_spent
CustomerID,
14646,280206.02
18102,259657.30
17450,194550.79
16446,168472.50
14911,143825.06
...,...
17956,12.75
16454,6.90
14792,6.20


In [18]:
# To calculate the 95th percentile we can do the following, first multiply the amount of customers by 95%

quantile = 0.95*4339
print(quantile)


4122.05


In [25]:
# Now, if we order in ascending order the customers by their expense, we now that the ones fallin in the highest 216 posittions,
# (4339 - 4122.05 = 216.95 ~ we can round to 216 highest positions ~) will be situated above the 95% of customers with most expense.

sorted_agg_expense = agg_expense.sort_values('amount_spent', ascending=False)
sorted_agg_expense.reset_index(inplace=True)
sorted_agg_expense.head(217)

# Sorting alphabetically the list and retrieving the first 216 values we can have the list of the customers above 95th quantile in spent
# amount of money.


,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06
...,...,...
212,14505,5905.64
213,14415,5900.28
214,14944,5900.25
215,13209,5891.61


In [60]:
# There is a quicker way with the .quantile() function. As it can be seen below, it will retrieve us the amount 
# spent that stablishes the limit of the 95th quantile. More than this amount of money will be considered above 
# 95th quantile, same for the 75th quantile. 

agg_expense.quantile(0.95)
agg_expense.quantile(0.75)

amount_spent    1661.64
Name: 0.75, dtype: float64

In [61]:
# Finally, since we know where's the limit of the 95th quantile, we can create a function to add a new row
# and depending on the amount spent give VIP status or not.

def customer_status(row):
    if row['amount_spent'] >=5870:
        return 'VIP'
    elif 5870 >= row['amount_spent'] >= 1662:
        return 'Preferred'
    else:
        return 'Standard'

sorted_agg_expense['Status'] = sorted_agg_expense.apply(customer_status, axis=1)
sorted_agg_expense

# With this results, we have a list with the VIP and Preferred customers sorted, and with their customer ID's,
# if we want, we can do an inner join with the orders table and add the status columns to know which of our 
# orders have been done by VIP or Preferred customers. We can also keep this dataframe for more insights.

,CustomerID,amount_spent,Status
0,14646,280206.02,VIP
1,18102,259657.30,VIP
2,17450,194550.79,VIP
3,16446,168472.50,VIP
4,14911,143825.06,VIP
...,...,...,...
4334,17956,12.75,Standard
4335,16454,6.90,Standard
4336,14792,6.20,Standard
4337,16738,3.75,Standard


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [62]:
# We can do an inner join with the orders dataframe and our new sorted dataframe to add the countries
# of the customers by their ID.

customers_procedence = sorted_agg_expense.merge(orders[['Country','CustomerID']],
                                how = 'inner',
                                left_on = 'CustomerID',
                                right_on = 'CustomerID')

customers_procedence.drop_duplicates(inplace=True)
customers_procedence.reset_index(inplace=True)
customers_procedence

#There is some extra rows but can be due to some data not properly clean from orders dataframe when merging both.

,index,CustomerID,amount_spent,Status,Country
0,0,14646,280206.02,VIP,Netherlands
1,2080,18102,259657.30,VIP,United Kingdom
2,2511,17450,194550.79,VIP,United Kingdom
3,2848,16446,168472.50,VIP,United Kingdom
4,2851,14911,143825.06,VIP,EIRE
...,...,...,...,...,...
4342,397917,17956,12.75,Standard,United Kingdom
4343,397918,16454,6.90,Standard,United Kingdom
4344,397920,14792,6.20,Standard,United Kingdom
4345,397922,16738,3.75,Standard,United Kingdom


In [63]:
# Now we can create a filter for the VIP customers and count the number of times that countries repeat.

condition = customers_procedence['Status'] == 'VIP'

VIP_cust = customers_procedence[condition]
VIP_cust['Country'].value_counts()

# We can see that the country with the most VIP customers is United Kingdom

United Kingdom     177
Germany             10
France               9
Switzerland          3
Portugal             2
Japan                2
Australia            2
EIRE                 2
Spain                2
Cyprus               1
Netherlands          1
Channel Islands      1
Finland              1
Singapore            1
Norway               1
Denmark              1
Belgium              1
Sweden               1
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [68]:
# We can retrieve that from the 'customers_procedence' dataframe adding an extra filter to the condition
condition = (customers_procedence['Status'] == 'VIP') | (customers_procedence['Status'] == 'Preferred')

VIPnPref_cust = customers_procedence[condition]
VIPnPref_cust['Country'].value_counts()

# Once again, as we can see below, United Kingdom is the country with the most VIP and Preferred customers
# with 931 customers.

United Kingdom     931
Germany             39
France              29
Belgium             12
Spain                9
Switzerland          9
Portugal             7
Norway               7
Italy                5
Finland              5
Australia            4
Japan                4
Channel Islands      4
Cyprus               4
Denmark              3
EIRE                 3
Austria              3
Israel               2
Sweden               2
Iceland              1
Poland               1
Greece               1
Malta                1
Canada               1
Singapore            1
Netherlands          1
Lebanon              1
Name: Country, dtype: int64